
![ImageName](pic/title.jpg)  

**作者：[lqy](https://www.heywhale.com/home/user/profile/5f27fd9633e1be002cc65a1d)** 华东师范大学气象学研究生、和鲸社区气象数据科学频道版主

🐋：本项目来自和鲸社区《[气象训练营⑦：WRF模式后处理](https://www.heywhale.com/home/competition/64478fec113e81a18dc70cd1)》活动，所有教案代码都可以一键跑通，你可以 fork 后在线在线运行、调试学习、完成作业练习。  

学习过程中如果你遇到任何问题，欢迎使用搜索引擎，或在 [讨论区](https://www.heywhale.com/home/competition/forumlist/64478fec113e81a18dc70cd1) 中发帖提出，我们很乐意为你提供帮助。

# 关卡三：绘制WRF模拟的风场

## 一、风速数据有何不同

在上一关卡中，我们介绍了**瞬时变量和累积变量**的差异，与数据所表示的时间有关。当然，WRF数据还可以分为**矢量数据和标量数据**。矢量数据是包含运动方向和大小的，标量数据则是只有大小。其中，大部分变量都是标量数据，而风速是一个很典型的矢量数据。不仅如此，**风速可以用U风速（纬向）和V风速（经向），也可以用风速大小和风向大小来加以表示**，具有很大的灵活性。  
  
此外，**风速是在环流场分析中必不可少的变量**，在业务和科研中都被广泛使用。

## 二、提取WRF模拟的风速数据

不同于降雨量，**风速不仅有近地面风速、还有高空风速，具有高度层信息**。  
  
**变量名怎么找？就是关卡1提到的这个网址：https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.getvar.html**  
  
get_var可以直接提取如下变量：  
  
**近地面风速（模式输出的通常是10m高度）**：  
+ wspd_wdir10：网格坐标上的10米风速在风向（10m Wind Speed and Direction (wind_from_direction) in Grid Coordinates）  
+ uvmet10_wspd_wdir：地理坐标上的10米风速在风向（10m Wind Speed and Direction (wind_from_direction) Rotated to Earth Coordinates）  
  
**不同高度层的风速**：  
+ ua：纬向风速（U-component of Wind on Mass Points）  
+ va：经向风速（V-component of Wind on Mass Points）  
  
风速这个变量有点特殊，它跟坐标表示方式有关，**有网格坐标和地理坐标两种提取方式**。  
  
下面，我们来看看这两种提取方式存在差异吗？

In [ ]:
# WRF数据目录
wrfout_path = './dataset/wrfout/'

In [ ]:
# 导入模块
from netCDF4 import Dataset
from wrf import getvar

### 1. 网格坐标上的风速

网格坐标上的风速即 WRF 模拟的每个格点上的风速大小及风向

In [ ]:
wrf_file = Dataset(wrfout_path + 'wrfout_d01_2019-08-09_06_00_00')
# 提取网格坐标上的风速
wspd_wdir10 = getvar(wrf_file, 'wspd_wdir10', timeidx=0)
wspd_wdir10

In [ ]:
# 快速预览网格坐标上的风速大小
wspd10 = wspd_wdir10[0]
wspd10.plot()

### 2. 地理坐标上的风速

由于 WRF 模拟的地图投影设置可能与标准经纬度网格存在差异，而在地理坐标系上风速的表达与原始的格点模拟之间会存在一定的差异，因此，在地图上绘制风速的时候需要进行适当地转换。

In [ ]:
# 提取地理坐标上的风速
uvmet10_wspd_wdir = getvar(wrf_file, 'uvmet10_wspd_wdir', timeidx=0)
uvmet10_wspd_wdir

In [ ]:
# 快速预览地理坐标上的风速
uvmet10_wspd = uvmet10_wspd_wdir[0]
uvmet10_wspd.plot()

### 3. 两种风速表达的差异

乍一看，没啥区别，但真的完全一样吗？我们通过作差的方式，绘制一下两者差异的分布，可以看出还是存在一定差异的，但是数值在1e-6，远远小于我们风速本身的数值。

In [ ]:
# 绘制wspd10和uvmet10_wspd的差异
(wspd10-uvmet10_wspd).plot()

尽管风速上的差异数量级非常小，但还存在风向上的差异。我们通常都是在地理坐标系上画图的，因此用`uvmet10_wspd_wdir`更合适些。  

![Image Name](pic/wspd.png)  

如果想深入了解两者的差异，推荐阅读：https://www-k12.atmos.washington.edu/~ovens/wrfwinds.html  

### 4. 网格坐标风速转为地理坐标风速

使用`uvmet10_wspd_wdir`可以直接提取地理坐标上的风速，但是我们想要自己实现网格坐标风速到地理坐标风速上的转换，需要怎么实现呢？  
> **uvmet函数：https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.uvmet.html**

In [ ]:
# 导入模块
from wrf import uvmet
# 获取网格坐标上的1Om高度的UV风
u10 = getvar(wrf_file, 'U10', timeidx=0)
v10 = getvar(wrf_file, 'V10', timeidx=0)
# 计算地理坐标系上的1Om高度的UV风
cen_lat, cen_lon = wrf_file.CEN_LAT, wrf_file.CEN_LON
uvmet10 = uvmet(u10, v10, u10.XLONG, u10.XLAT, cen_lon, cone=1, meta=True, units='m s-1')
uvmet10

In [ ]:
# 纬向风的快速预览
uvmet10[0].plot()

In [ ]:
# 经向风的快速预览
uvmet10[1].plot()

**u为正，表示西风，从西边吹来的风。v为正，表示南风，从南边从来的风。**

In [ ]:
# 风向的快速预览
uvmet10_wspd_wdir[1].plot()

**风向是按正北方向起算的，0度表示北风，90度表示东风。**  
  
![Image Name](pic/fx.png)  


## 三、如何计算风速数据

风速是一个矢量，不仅包含大小，还具有方向。因此，**计算风速既涉及大小的计算，又涉及方向的计算**。

### UV分量转为风速风向

> **metpy.calc.wind_direction()：https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.wind_direction.html**  
    
    
![Image Name](pic/wind_direction.png)    
    
    
**我们使用wind_direction()时需要注意参数convention**，当convention默认为from时，它遵循的是气象惯例，当convention默认为to时，它遵循的是海洋惯例。由于海洋惯例和气象惯例是相反的，所以在使用的过程中需要注意。

In [ ]:
# 提取 WRF 模拟的三维风场，通过[0]进行索引仅提取其中一个高度层的uv风速
ua = getvar(wrf_file, 'ua', timeidx=0)[0]
va = getvar(wrf_file, 'va', timeidx=0)[0]

In [ ]:
ua

In [ ]:
va

观察 `ua` 和 `va` 两个 DataArray ，其 `Attribute` （属性） 中包含了物理单位 `units : m s-1`。这样我们就能用 `metpy` 直接进行计算，无需再申明物理单位。

In [ ]:
# 导入模块
import metpy.calc as mpcalc
# 根据 uv 分量计算风速
ws = mpcalc.wind_speed(ua, va)
ws

### ✍ 小练习：比较convention参数对风向计算的影响  

convention参数分别设置为`from`和`to`，比较风向计算的结果差异

In [ ]:
### （你的代码）###



### ✍ 小练习：风速风向转为UV分量  

提示：可以使用[metpy.calc.wind_components](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.wind_components.html)函数实现。

In [ ]:
### （你的代码）###



## 四、绘制风速数据的三种形式

由于风速数据的特殊性和复杂性，我们的可视化方式也更为多样，可以用**箭头图（矢量图）、风羽图（风杆图）、流线图**三种形式~

### 形式1：箭头图（矢量图）

箭头图（矢量图）是非常容易直观，大众普遍都能接受的一种表示风场的形式。**箭头大小表示风速大小，箭头方向表示风速方向。**  
  
Matplotlib提供了plt.quiver和plt.quiverkey两个函数，前者用于画箭头，后者用于标注箭头图例（类似于比例尺的效果），Cartopy中也继承了这两个方法。  
  
> **plt.quiver()：https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html**    
> **plt.quiverkey()：https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiverkey.html**    
   
   
控制箭头的重要参数：  
![Image Name](pic/control_arrow.png)    
  
  
推荐阅读：[炸鸡人博客 | Matplotlib 系列：图解 quiver](https://zhajiman.github.io/post/matplotlib_quiver/)

In [ ]:
# 导入模块
from wrf import to_np, getvar, uvmet, latlon_coords, geo_bounds, get_cartopy, cartopy_xlim, cartopy_ylim
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr
from metpy.units import units
from metpy.calc import height_to_geopotential, geopotential_to_height
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.feature import NaturalEarthFeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import shapely.geometry as sgeom
import cmaps
from glob import glob
from copy import copy

In [ ]:
'''
Andrew Dawson 提供了解决方法: https://gist.github.com/ajdawson/dd536f786741e987ae4e
'''

def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])

def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])

def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible: 
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
# 提取WRF模拟的经纬度数组
lats, lons = latlon_coords(u10)
# 提取WRF模拟的投影设置
wrf_proj = get_cartopy(u10)

In [ ]:
# 创建画布
fig = plt.figure(figsize=(10,8))
# 设置地图投影
ax = plt.axes(projection=wrf_proj)
# 设置地图范围
ax.set_xlim(cartopy_xlim(u10))
ax.set_ylim(cartopy_ylim(u10))

# 读取国界线
province = shpreader.Reader('./dataset/中国地图/China_provinces/China_provinces.shp')
# 读取九段线
nineline = shpreader.Reader('./dataset/中国地图/China_10-dash_line/China_10-dash_line.shp')
# 绘制国界线
ax.add_geometries(province.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5, 
                  edgecolor='k',
                  facecolor='none',
                  zorder=2)
# 绘制九段线
ax.add_geometries(nineline.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5,
                  color='k',
                  zorder=2)

# 绘制风速值（pcolormesh方法进行格点绘制）
im = ax.pcolormesh(to_np(lons), 
                   to_np(lats), 
                   to_np(ws), 
                   vmin=0, 
                   vmax=60, 
                   cmap=cmaps.MPL_BuPu, 
                   transform=ccrs.PlateCarree())

# 为风速值添加colorbar
cbar = plt.colorbar(im, ax=ax, extend='max', shrink=1)
cbar.set_label('Windspeed (m/s)', fontdict={'size':20})
cbar.ax.tick_params(labelsize=20)

# 绘制箭头
Q = ax.quiver(to_np(lons), to_np(lats), 
              to_np(ua), to_np(va), 
              pivot='middle', 
              transform=ccrs.PlateCarree(), 
              regrid_shape=20)

# 绘制箭头图例
qk = ax.quiverkey(Q, 
                  0.8, 1.02,                  
                  30, '30 m/s', 
                  labelpos='E',
                  coordinates='axes',
                  fontproperties={'size':15})

# 添加经纬度网格线
ax.gridlines(color='black', linestyle='dotted')
# 添加经纬度标签
xticks = list(np.arange(100,140,2))
yticks = list(np.arange(20,40,2))
fig.canvas.draw()
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)

# 添加标题
plt.title(str(u10.Time.values)[0:16]+'(UTC)', loc='left', fontsize=15)
plt.show()

### ✍小练习：调整regrid_shape参数

Cartopy 还提供了一个非常便利的参数`regrid_shape`，可以将矢量场重新插值到投影坐标系中的规则网格上，以达到**规整矢量位置或稀疏箭头密度的目的**。  
> 参考：https://scitools.org.uk/cartopy/docs/latest/gallery/vector_data/regridding_arrows.html  
  
![Image Name](pic/regrid_shape.png)  


### 形式二：风羽图（风杆图）

风羽图（风杆图）是一种专业的风场绘制方式，在论文里会经常出现，在业务中也有广泛应用。但需要注意的是**风杆表示的风速大小**。

In [ ]:
# 创建画布
fig = plt.figure(figsize=(10,8))
# 设置地图投影
ax = plt.axes(projection=wrf_proj)
# 设置地图范围
ax.set_xlim(cartopy_xlim(u10))
ax.set_ylim(cartopy_ylim(u10))

# 读取国界线和九段线
province = shpreader.Reader('./dataset/中国地图/China_provinces/China_provinces.shp')
nineline = shpreader.Reader('./dataset/中国地图/China_10-dash_line/China_10-dash_line.shp')
# 绘制国界线和九段线
ax.add_geometries(province.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5, 
                  edgecolor='k',
                  facecolor='none',
                  zorder=2)
ax.add_geometries(nineline.geometries(), 
                  crs=ccrs.PlateCarree(), 
                  linewidth=0.5,
                  color='k',
                  zorder=2)

# 绘制风速值
im = ax.pcolormesh(to_np(lons), 
                   to_np(lats), 
                   to_np(ws), 
                   vmin=0, 
                   vmax=60, 
                   cmap=cmaps.MPL_BuPu, 
                   transform=ccrs.PlateCarree())

# 为风速值添加colorbar
cbar = plt.colorbar(im, ax=ax, extend='max', shrink=1)
cbar.set_label('Windspeed (m/s)', fontdict={'size':20})
cbar.ax.tick_params(labelsize=20)


# 绘制风羽
# ::25表示数据抽稀，每隔25个格点绘制风羽
# dict(half=2, full=4, flag=20)表示短线代表风速2m/s，长线代表风速4m/s，旗帜代表风速20m/s
ax.barbs(to_np(lons)[::25,::25], to_np(lats)[::25,::25], 
         to_np(ua)[::25,::25], to_np(va)[::25,::25], 
         length=8, 
         linewidth=0.5, 
         barb_increments=dict(half=2, full=4, flag=20),
         transform=ccrs.PlateCarree())


# 添加经纬度网格线
ax.gridlines(color='black', linestyle='dotted')
# 添加经纬度标签
xticks = list(np.arange(100,140,2))
yticks = list(np.arange(20,40,2))
fig.canvas.draw()
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)

# 添加标题
plt.title(str(u10.Time.values)[0:16]+'(UTC)', loc='left', fontsize=15)
plt.show()

#### 小练习：自定义风羽

`barb_increments`参数非常重要，直接决定了风羽所表示的风速大小。  
  
`dict(half=2, full=4, flag=20)`表示**短线代表风速2m/s，长线代表风速4m/s，旗帜代表风速20m/s**。  
  
  
但是，如果我们想设置4类属性，例如：短线代表风速2m/s，长线代表风速4m/s，空心三角代表风速20m/s，实心三角代表风速50m/s。这样是否可以实现呢？  
  
> 推荐阅读：[Matplotlib风羽自定义](https://www.cnblogs.com/kallan/p/6279932.html) 

In [ ]:
### （你的代码）###



### 形式三：流线图

matplotlib绘图要求数据是列表或者`numpy.array()`格式。`xarray.Dataarray()`格式可能会在`steamplot`、`barbs`、`quiver`等绘制中报错，需要用**values或者to_np函数**将数据转为`numpy.array`格式。  
  
> **streamplot函数：https://matplotlib.org/stable/gallery/images_contours_and_fields/plot_streamplot.html**  


![Image Name](pic/lxt.png)  


流线图的绘制逻辑与之前的一样的，这部分作为小练习啦~~  


In [ ]:
### （你的代码）###



恭喜你完成了WRF后处理训练营第三关的学习材料，对风速数据有了更深入的认识，**掌握UV分量与风速风向之间转换**，并能够用**不同形式（风羽图、箭头图、流线图）** 来展示WRF模拟的风速数据。

## 闯关题  

### STEP1：根据要求完成题目

1.WRF模拟数据中的 **风**数据属于？  
A. 矢量，瞬时量  
B. 矢量，累积量  
C. 标量，瞬时量  
D. 标量，累积量  
  
2.绘制台风利奇马在**对流层低层和高层的风场**，它们分别是？  
A. 低层顺时针，高层顺时针  
B. 低层逆时针，高层逆时针  
C. 低层逆时针，高层顺时针  
D. 低层顺时针，高层逆时针  
  
3.从WRF模拟初始时刻2019-08-08_18_00_00（UTC）到2019-08-09_06_00_00（UTC），台风利奇马10m风速的最大值出现在哪一时次？（答案请转换为北京时间，格式例如：'08-09_06'）

In [ ]:
# 填入你的答案
answer_1 = ''
answer_2 = ''
answer_3 = '' #格式例如：'08-09_06'

## ⚡ 下一关预告：  

下一关我们将介绍如何计算WRF模拟的衍生变量，得到你最感兴趣的模拟变量~~